In [1]:
import os
import sys
sys.path.append('../train')
sys.path.append('../')

import numpy as np
import torch


import training_utils
from torch.utils.data import *
from my_dataloader import *
from read_data import *

from model_bank import *
from models import *

import score

In [2]:
# model_path = '/Lun2/rzz/kaldi-master/egs/zhiyong/sdsvc_exp/train_exp/new_pipe_test_fullmix 1L/ckpt/vox1test_metric_saver_MINC.model'
# model_id = 'Xvector_SAP_1L'
# model_metric = 'AM_normfree_softmax_anneal_ce_head'

In [3]:
# model_path = '/Lun2/rzz/kaldi-master/egs/zhiyong/sdsvc_exp/train_exp/new_pipe_test_fullvoxonly/ckpt/vox1test_metric_saver_MINC.model'
# model_id = 'Xvector_SAP_1L'
# model_metric = 'AM_normfree_softmax_anneal_ce_head'

In [4]:
model_path = '/Lun2/rzz/kaldi-master/egs/zhiyong/sdsvc_exp/train_exp/full mix 1L FOCAL_ALDA_OPT e_lr2/ckpt/sdsvc_metric_saver_MINC.model'
model_id = ''
model_metric = ''

In [5]:
class Config(object):

    train_list = '/Lun0/zhiyong/sdsvc_libri_full_dataset/mix_8983_fulltrain_fixed.csv'
    vox_val_list = '/Lun0/zhiyong/sdsvc_libri_full_dataset/source_vox_7323_fullval.csv'
    libri_val_list = '/Lun0/zhiyong/sdsvc_libri_full_dataset/source_libri_1172_fullval.csv'
    sdsvc_val_list = '/Lun0/zhiyong/sdsvc_libri_full_dataset/target_sdsvc_488_fullval.csv'
    sdsvc_trial_list = '/Lun0/zhiyong/sdsvc_libri_full_dataset/sdsvc_trial_list.csv'
    sdsvc_trial_keys = '/Lun0/zhiyong/sdsvc_libri_full_dataset/sdsvc_trial_keys.csv'
    vox1test_trial_list = '/Lun0/zhiyong/sdsvc_libri_full_dataset/vox1test_trial_list.csv'
    vox1test_trial_keys = '/Lun0/zhiyong/sdsvc_libri_full_dataset/vox1test_trial_keys.csv'
    libri_trial_list = '/Lun0/zhiyong/sdsvc_libri_full_dataset/libri_trial_list.csv'
    libri_trial_keys = '/Lun0/zhiyong/sdsvc_libri_full_dataset/libri_trial_keys.csv'
    # vox1test_aux_list = '/home/great10/sdsvc_libri_full_dataset/vox1test_aux_list.csv'
    # vox1test_aux_keys = '/home/great10/sdsvc_libri_full_dataset/vox1test_aux_keys.csv'

    scoring_config = {'p_target': [0.01], 'c_miss': 10, 'c_fa': 1}

    num_workers = 32  # how many workers for loading data


In [6]:
opt = Config()
total_step = 0

In [7]:
# model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 7811, \
# 's': 50, 'm': 0.2, 'anneal_steps': 10000}
# torch.backends.cudnn.benchmark = False

# os.environ['CUDA_VISIBLE_DEVICES'] = str(3)

# device = torch.device("cuda:0")

# model = get_model(model_id, model_metric, model_settings, None)
# checkpoint = torch.load(model_path, map_location='cpu')
# model.load_state_dict(checkpoint['model'], strict=True)
# model = model.to(device)

In [8]:
model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 7811, 'source_class_num': 7323, \
's': 50, 'm': 0.2, 'anneal_steps': 10000, 'weight': 15, 'focal_d_gamma': 0, 'focal_al_gamma': 0}
torch.backends.cudnn.benchmark = False

os.environ['CUDA_VISIBLE_DEVICES'] = str(3)

device = torch.device("cuda:0")

model = FOCAL_ALDA.FOCAL_ALDA_2DO_OPT_FAST(model_settings)
checkpoint = torch.load(model_path, map_location='cpu')
model.load_state_dict(checkpoint['model'], strict=True)
model = model.to(device)

In [9]:
# torch.backends.cudnn.benchmark = False
# model.eval()

# train_data = PickleDataSet(opt.vox1test_trial_list)
# train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
# batch_sampler=None, num_workers=opt.num_workers, collate_fn=None,\
# pin_memory=False, drop_last=False, timeout=0,\
# worker_init_fn=None, multiprocessing_context=None)

# test_list = {}

# for count, (batch_x, batch_y) in enumerate(train_dataloader):
#     batch_x = batch_x.to(device)
#     label = batch_y[0]

#     batch_y = torch.tensor([0]).to(device)

#     with torch.no_grad():
#         _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

#     emb = emb.squeeze().data.cpu().numpy()

#     if label not in test_list.keys():
#         test_list[label] = emb[None, :]
#     else:
#         print('repeat eer:', label)
#         break

# msg = "vox1test_ASV_eval Step: {:} Embcount: {:}".format(total_step, (count + 1))
# print(msg)

# out_dir = './'
# f_out = open(os.path.join(out_dir, 'scores'), 'w')   

# for i in test_list:
#     test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

# with open(opt.vox1test_trial_keys, 'r') as f:
#     for count, line in enumerate(f):
#         if count == 0:
#             pass
#             # print(line)

#         enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
#         test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

#         cosine = np.dot(enroll_emb, test_emb)

#         f_out.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')

# f_out.close()

# msg = "vox1test_ASV_eval Step: {:} Trialcount: {:}".format(total_step, (count + 1))
# print(msg)


# eer, minc, actc = score.scoring(os.path.join(out_dir, 'scores'), opt.vox1test_trial_keys, opt.scoring_config)

# current_lr = 0.0
# msg = "vox1test_ASV_eval Step: {:} EER: {:.4f} MINC: {:.4f} ACTC: {:.4f} Lr: {:.5f}"\
# .format(total_step, eer, minc, actc, current_lr)
# print(msg)
   

In [10]:
# torch.backends.cudnn.benchmark = False
# model.eval()

# train_data = PickleDataSet(opt.sdsvc_trial_list)
# train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
# batch_sampler=None, num_workers=opt.num_workers, collate_fn=None,\
# pin_memory=False, drop_last=False, timeout=0,\
# worker_init_fn=None, multiprocessing_context=None)

# test_list = {}

# for count, (batch_x, batch_y) in enumerate(train_dataloader):
#     batch_x = batch_x.to(device)
#     label = batch_y[0]

#     batch_y = torch.tensor([0]).to(device)

#     with torch.no_grad():
#         _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

#     emb = emb.squeeze().data.cpu().numpy()

#     if label not in test_list.keys():
#         test_list[label] = emb[None, :]
#     else:
#         print('repeat eer:', label)
#         break        

# msg = "sdsvc_ASV_eval Step: {:} Embcount: {:}".format(total_step, (count + 1))
# print(msg)

# out_dir = './'
# f_out = open(os.path.join(out_dir, 'scores'), 'w')   

# for i in test_list:
#     test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

# with open(opt.sdsvc_trial_keys, 'r') as f:
#     for count, line in enumerate(f):
#         if count == 0:
#             pass
#             # print(line)

#         enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
#         test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

#         cosine = np.dot(enroll_emb, test_emb)

#         f_out.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')

# f_out.close()

# msg = "sdsvc_trial_keys Step: {:} Trialcount: {:}".format(total_step, (count + 1))
# print(msg)

# eer, minc, actc = score.scoring(os.path.join(out_dir, 'scores'), opt.sdsvc_trial_keys, opt.scoring_config)

# current_lr = 0.0
# msg = "sdsvc_ASV_eval Step: {:} EER: {:.4f} MINC: {:.4f} ACTC: {:.4f} Lr: {:.5f}"\
# .format(total_step, eer, minc, actc, current_lr)
# print(msg)

In [11]:
torch.backends.cudnn.benchmark = False
model.eval()

train_data = PickleDataSet(opt.libri_trial_list)
train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
batch_sampler=None, num_workers=opt.num_workers, collate_fn=None,\
pin_memory=False, drop_last=False, timeout=0,\
worker_init_fn=None, multiprocessing_context=None)

test_list = {}

for count, (batch_x, batch_y) in enumerate(train_dataloader):
    batch_x = batch_x.to(device)
    label = batch_y[0]

    batch_y = torch.tensor([0]).to(device)

    with torch.no_grad():
        _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

    emb = emb.squeeze().data.cpu().numpy()

    if label not in test_list.keys():
        test_list[label] = emb[None, :]
    else:
        print('repeat eer:', label)
        break        

msg = "libri_ASV_eval Step: {:} Embcount: {:}".format(total_step, (count + 1))
print(msg)

out_dir = './'
f_out = open(os.path.join(out_dir, 'scores'), 'w')   

for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

with open(opt.libri_trial_keys, 'r') as f:
    for count, line in enumerate(f):
        if count == 0:
            pass
            # print(line)

        enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
        test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

        cosine = np.dot(enroll_emb, test_emb)

        f_out.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')

f_out.close()

msg = "libri_trial_keys Step: {:} Trialcount: {:}".format(total_step, (count + 1))
print(msg)

eer, minc, actc = score.scoring(os.path.join(out_dir, 'scores'), opt.libri_trial_keys, opt.scoring_config)

current_lr = 0.0
msg = "libri_ASV_eval Step: {:} EER: {:.4f} MINC: {:.4f} ACTC: {:.4f} Lr: {:.5f}"\
.format(total_step, eer, minc, actc, current_lr)
print(msg) 

libri_ASV_eval Step: 0 Embcount: 2703
libri_trial_keys Step: 0 Trialcount: 32436
libri_ASV_eval Step: 0 EER: 0.0220 MINC: 0.1265 ACTC: 1.0000 Lr: 0.00000
